In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from utilities import technical_analysis
from utilities import generate_candlestick_patterns
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import os
import talib
from dotenv import load_dotenv
from sqlalchemy import text


In [ ]:


load_dotenv()

with open(os.getenv("PRIVATE_KEY_PATH"), "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

private_key_pkcs8 = base64.b64encode(
    private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
).decode("utf-8")  # ✅ base64 string, not bytes!

engine = create_engine(URL(
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    user=os.getenv("SNOWFLAKE_USER"),
    private_key=private_key_pkcs8,
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE")
))

with engine.connect() as conn:
    result = conn.execute(text("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_TIMESTAMP();"))
    for row in result:
        print("✅ Snowflake connected:", row)


In [ ]:
from onchain import fetch_all_blockchain_metrics
df_blockchain = fetch_all_blockchain_metrics()
df_blockchain[['hash_rate','miner_revenue','active_addresses','difficulty']] = df_blockchain[['hash_rate','miner_revenue','active_addresses','difficulty']].ffill()
# Fill all numeric NaNs with the column mean
df_blockchain = df_blockchain.apply(
    lambda col: col.fillna(col.mean()) if pd.api.types.is_numeric_dtype(col) else col
)
# Repeat daily values over each 15-min slot (divide value into 96 chunks)
df_normalized = df_blockchain.resample("1h").ffill()
df_normalized[df_blockchain.columns[0]] = df_normalized[df_blockchain.columns[0]] / 24

In [ ]:
df_normalized.reset_index(inplace=True)
df_normalized['date']=df_normalized['timestamp']
df_normalized.drop(columns=(['timestamp']),inplace=True)



In [ ]:
query = """
SELECT  
    btc.date,
    btc.open,
    btc.close,
    btc.high,
    btc.low,
    btc.volume, 
    nasd.close AS NASDAQ, 
    sp500.close AS SP500, 
    vix.close AS VIX,
    dwj.close AS DOWJONES,
    (
        SELECT min(close ) as close
        FROM CRYPTODB.PUBLIC.vw_btc 
        WHERE date = DATEADD(minute, 15, btc.date)
        
    ) AS Target
FROM CRYPTODB.PUBLIC.vw_btc btc
LEFT JOIN CRYPTODB.PUBLIC.vw_nasdaq nasd
    ON btc.date = nasd.timestamp
LEFT JOIN CRYPTODB.PUBLIC.vw_sp500 sp500
    ON btc.date = sp500.timestamp
LEFT JOIN CRYPTODB.PUBLIC.vw_vix vix
    ON btc.date = vix.timestamp
LEFT JOIN CRYPTODB.PUBLIC.vw_dowjones dwj
    ON btc.date = dwj.timestamp
WHERE btc.date >= '2015-01-01'
ORDER BY btc.date ASC
"""

data = pd.read_sql(query,con=engine)
data.head()

In [ ]:
df= pd.DataFrame()
df = data.copy()
df[['nasdaq', 'sp500', 'vix', 'dowjones']] = df[['nasdaq', 'sp500', 'vix', 'dowjones']].ffill()








In [ ]:
df = df.resample('1H', on='date').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    'nasdaq': 'last',
    'sp500': 'last',
    'vix': 'last',
    'dowjones': 'last',
    # Other numeric on-chain features: 'mean' or 'last'
})


In [ ]:
df.columns = [col.capitalize() for col in df.columns]

In [ ]:
df = pd.merge(df, df_normalized, on='date', how='left')

In [ ]:
from clusteranalysis import perform_kmeans_with_elbow_silhouette
from utilities import technical_analysis, generate_candlestick_patterns

def prepare_pipeline(df_raw):
    # Step 1: Add technical indicators
    df_ta = technical_analysis(df_raw.copy())

    # Step 2: Add candlestick pattern features
    df_patterns = generate_candlestick_patterns(df_ta)

    # Step 3: Perform clustering
    df_clustered = perform_kmeans_with_elbow_silhouette(df_patterns,plot=False)

    return df_clustered

In [ ]:
df.dropna(inplace=True)

In [ ]:
def classify_return(r, threshold=0.003):
    if r > threshold:
        return 1   # up
    elif r < -threshold:
        return -1  # down
    else:
        return 0   # neutral

In [ ]:
df_raw = df.copy()
df_output = prepare_pipeline(df_raw)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from clusteranalysis import perform_kmeans_with_elbow_silhouette


df_output['target_return'] = df_output['Close'].shift(-1) / df['Close'] - 1
df_output['target_class'] = df_output['target_return'].apply(classify_return).astype(int)

df_output['return_t-1'] = df['target_return'].shift(1)
df_output['rolling_vol'] = df['target_return'].rolling(10).std()

import numpy as np
df_output['log_return'] = np.log(df_output['Close'] / df['Close'].shift(1))  # past return (t-1)
df_output['log_return_t-1'] = df_output['log_return'].shift(1)
df_output['log_return_sum_3'] = df_output['log_return'].rolling(3).sum()

df_output['slope'] = df_output['Close'].diff(3)

# Your base features
numerical_features = ['Nasdaq', 'Volume','hash_rate','miner_revenue','active_addresses','difficulty','log_return','log_return_t-1','log_return_sum_3','slope','MACD',
                     'MACD_Signal','MFI','RSI','Bollinger_Upper','Bollinger_Lower','Stochastic']
categorical_features = [
   'pattern_label'
]

# Combine them
all_features = numerical_features + categorical_features

# Prepare the dataframe
df_model = df_output.dropna(subset=all_features + ['target_class'])

X = df_model[all_features]
y = df_model['target_class']

# One-hot encode categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)



In [ ]:




# Now you can plug this into your model:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, max_depth=10, class_weight='balanced', random_state=42)

#RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)

# Split BEFORE transforming
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False  # no shuffle for time series
)

# Fit-transform on training, transform on test
X_train = preprocessor.fit_transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# 📊 Model Evaluation Metrics Explained:
#
# - Accuracy: Overall, how often the model is correct.
#             (Good for balanced datasets, less useful when classes are imbalanced)
#
# - Precision: Of all predicted "Up" signals (class 1), how many were actually correct?
#              High precision = fewer false positive trades (i.e., false buy signals).
#
# - Recall: Of all actual "Up" moments in the market, how many did we successfully detect?
#           High recall = fewer missed opportunities (captures more real upward moves).
#
# - F1 Score: The harmonic mean of Precision and Recall.
#             A balanced measure useful when we care about both false positives and false negatives.
#
# In trading contexts:
# 👉 Precision favors capital preservation (less false buys).
# 👉 Recall favors opportunity capture (more trades, but possibly riskier).


In [ ]:
import plotly.graph_objects as go
import pandas as pd
from sklearn.metrics import confusion_matrix

# Replace these with your actual predictions and labels
labels = [-1, 0, 1]
label_names = ['Down (-1)', 'Neutral (0)', 'Up (1)']
cm = confusion_matrix(y_test, y_pred, labels=labels)
df_cm = pd.DataFrame(cm, index=label_names, columns=label_names)

# Create a heatmap
fig = go.Figure(data=go.Heatmap(
    z=df_cm.values,
    x=label_names,
    y=label_names,
    colorscale='Blues',
    colorbar=dict(title='Count')
))

# Add annotations
for i in range(df_cm.shape[0]):
    for j in range(df_cm.shape[1]):
        fig.add_annotation(
            x=label_names[j],
            y=label_names[i],
            text=str(df_cm.iloc[i, j]),
            showarrow=False,
            font=dict(color="black")
        )

fig.update_layout(
    title='Confusion Matrix (Multiclass)',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    width=700,
    height=500
)

fig.show()



In [ ]:
importances = model.feature_importances_
feature_names = preprocessor.get_feature_names_out()
sorted_idx = importances.argsort()[::-1]

print("🔍 Top 15 Most Important Features:")
for idx in sorted_idx[:15]:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")


